# Regularización

El propósito de la regularización es **evitar modelos sobreajustados modificando el comportamiento de descenso por gradiente, objetivo y datos.**

La técnica básica para regularizar es evaluar la bondad de cualquier modificación mediante estimación del rendimiento teórico (en validación). Esto se puede hacer mediante diversos métodos.

Cargamos el dataset como se ha hecho en prácticas anteriores

In [2]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; import keras_tuner

keras.utils.set_random_seed(23); input_dim = (28, 28, 1); num_classes = 10
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_val = x_train_val.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train_val = np.expand_dims(x_train_val, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train_val.shape, y_train_val.shape, x_test.shape, y_test.shape)
y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]

2024-11-28 19:29:46.660910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 19:29:46.673085: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 19:29:46.676500: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)


Esta vez el hipermodelo incorpora capas convolucionales y de pooling.

Two convolutional layers with:

- Filters: filters in the first layer, 2*filters in the second.
- Kernel size: (3, 3)
- Activation: ReLU

Max pooling layers reduce spatial dimensions by (2, 2).

Esto es equivalente a la anterior práctica. Sin embargo esta vez tenemos también capas encargadas de rotar, trasladar, y hacer zoom a las entradas:

```python
    M.add(keras.layers.RandomRotation(factor, fill_mode="nearest"))
    M.add(keras.layers.RandomTranslation(factor, factor, fill_mode="nearest"))
    M.add(keras.layers.RandomZoom(factor, fill_mode="nearest"))
    M.add(keras.layers.Rescaling(1./255))

In [3]:
class MyHyperModel(keras_tuner.HyperModel):
  def build(self, hp):
    M = keras.Sequential()
    M.add(keras.Input(shape=(28, 28, 1)))
    factor = hp.Float("factor", min_value=0.01, max_value=0.3, step=2, sampling="log")
    M.add(keras.layers.RandomRotation(factor, fill_mode="nearest"))
    M.add(keras.layers.RandomTranslation(factor, factor, fill_mode="nearest"))
    M.add(keras.layers.RandomZoom(factor, fill_mode="nearest"))
    M.add(keras.layers.Rescaling(1./255))

    filters = 64
    M.add(keras.layers.Conv2D(filters, kernel_size=(3, 3), activation="relu"))
    M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    M.add(keras.layers.Conv2D(2*filters, kernel_size=(3, 3), activation="relu"))
    M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    M.add(keras.layers.Flatten())
    M.add(keras.layers.Dense(units=800, activation='relu'))

    # dropout = hp.Float("dropout", min_value=0.0, max_value=0.5, step=0.1)
    dropout = 0.5
    M.add(keras.layers.Dropout(dropout))
    M.add(keras.layers.Dense(10, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.00015)

    M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return M
  def fit(self, hp, M, x, y, xy_val, **kwargs):
    factor = 0.32; patience = 5

    reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=factor, patience=patience, min_delta=1e-4, min_lr=1e-5)

    early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=1e-5)

    kwargs['callbacks'].extend([reduce_cb, early_cb])
    return M.fit(x, y, batch_size=256, epochs=100, validation_data=xy_val, **kwargs)

El proceso de cómputo de este entrenamiento es muy pesado (por eso solo vamos a hacer una trial).

In [4]:
tuner = keras_tuner.BayesianOptimization(
MyHyperModel(), objective="val_accuracy", max_trials=1, executions_per_trial=1,
overwrite=True, directory="/tmp", project_name="MNIST")

In [5]:
tuner.search(x_train, y_train, (x_val, y_val))

Trial 1 Complete [00h 28m 34s]
val_accuracy: 0.8572999835014343

Best val_accuracy So Far: 0.8572999835014343
Total elapsed time: 00h 28m 34s


In [6]:
tuner.results_summary(num_trials=1)

Results summary
Results in /tmp/MNIST
Showing 1 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
factor: 0.04
Score: 0.8572999835014343


In [7]:
num_models = 1
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=num_models)
best_models = tuner.get_best_models(num_models=num_models)
for m in range(num_models):
  values = best_hyperparameters[m].values
  score = best_models[m].evaluate(x_test, y_test, verbose=0)
  print(f'Model {m}: Hyperparameters: {values!s} Loss: {score[0]:.4} Precisión: {score[1]:.2%}')

/home/yassin/anaconda3/envs/per/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model 0: Hyperparameters: {'factor': 0.04} Loss: 0.4045 Precisión: 85.46%
